# QNumerics Arrays session

This notebook contains examples for the [QNumerics 2024](https://qnumerics.org/) session on arrays.

## Array basics


In this section, we'll look at some basic operations on arrays, including extending and truncating them.

In [ ]:
a = [1, 2, 3, 4]

In [ ]:
push!(a, 5)

In [ ]:
pushfirst!(a, 0)

In [ ]:
pop!(a)

In [ ]:
a

In [ ]:
popfirst!(a)

In [ ]:
a

In [ ]:
b = splice!(a, 2:3)

In [ ]:
a

In [ ]:
append!(a, [5, 6, 7]) # how does this differ from push!

In [ ]:
c = vcat(a, [8, 9, 10])

In [ ]:
d = hcat(a, rand(1:5, length(a)))

In [ ]:
reverse!(a)

In [ ]:
ndims(d)

In [ ]:
size(d)

In [ ]:
size(d, 2)

In [ ]:
e = fill(2, (3, 4, 5))

In [ ]:
f = 1:3:20 # a range -- note the that the step size is the SECOND argument

In [ ]:
collect(f)

## Indexing

Now let's look at the various ways we can index arrays. Note the special reserved Julia keyword `end`. Julia arrays support indexing with:
  - integers
  - ranges
  - vectors of integers, vectors of booleans
  - and more!

In [ ]:
a = collect(1:10)

In [ ]:
a[1:2:3]

In [ ]:
a[5:end]

In [ ]:
a[[false, true, false, true, true, false, false, false, false, true]]

In [ ]:
a[[1, 2, 7, 3]]

In [ ]:
a[5] # what happens if you use an Int8(5) as the index?

In [ ]:
b = rand(4, 4)

In [ ]:
b[[1, 10, 5, 2]]

In [ ]:
b[1:3:end]

In [ ]:
c = rand(2, 2, 2)

In [ ]:
c[1:4:end]

In [ ]:
b[1, 1]

In [ ]:
b[2:3, 3:4]

In [ ]:
c[2, 1, 2]

In [ ]:
c[1, :, 2]

In [ ]:
# b[1, 2, 1, 1] # do you think this should work?

In [ ]:
collect(eachindex(c))

In [ ]:
collect(CartesianIndices(size(c)))

## Higher dimensional arrays

Julia has good support for `N`-dimensional arrays when `N > 2`. This is useful for representing tensors or transforming a vector back and forth to a tensor representation (for `einsum`, for example.)

In [ ]:
d = rand(3, 4, 5)

In [ ]:
permutedims(d, (3, 2, 1))

In [ ]:
strides(d)

In [ ]:
mapslices(sum, d; dims=(1, 2))

In [ ]:
e = rand(ComplexF64, 5, 5)

In [ ]:
transpose(e)

In [ ]:
adjoint(e)

## Broadcasting 🔈

Broadcasting is used to do elementwise operations on Julia arrays, and can help us avoid allocations and write simple, extensible code without 5 nested `for` loops. Let's see it in action.

In [ ]:
d = rand(ComplexF64, 3, 3)

In [ ]:
abs2.(d)

In [ ]:
d = rand(0:10, 3, 3)

In [ ]:
d .+ 2

In [ ]:
e = rand(0:10, 10)

In [ ]:
e[1:2:end] .-= 3

Now we'll compare "naive" broadcasting with broadcast fusion using the `@.` operator, which allows us to force all operations on a line to be broadcasted simultaneously and avoids multiple loop executions.

In [ ]:
no_fusion(a) = abs2.(a) - 3a + sqrt.(a)

In [ ]:
fusion(a) = @. abs2(a) - 3a + sqrt(a)

We invoke each of our new functions once to have Julia and LLVM compile them for the provided types. Then we can do a fair comparison of their runtimes and allocations.

In [ ]:
fusion([1.1, 2.2]); #compile

In [ ]:
no_fusion([1.1, 2.2]); #compile

Let's use an input array large enough to see a real difference. We use the `@timev` macro to get more detailed information on what Julia was doing during the code execution -- this is a very useful tool for understanding why code is slow or fast!

In [ ]:
a = rand(Float64, 10^6);

In [ ]:
@timev no_fusion(a);

In [ ]:
@timev fusion(a);

## Views 👀

Julia's views provide a non-allocating window into an array. This lets us perform operations on array slices without thrashing the garbage collector (yay!). Again we'll write a function using views, and one that doesn't use views, and compare them. *Be careful* to use views only when you know you are not mutating the underlying array or if you intend to mutate it!

In [ ]:
a = rand(1:5, 10)

In [ ]:
b = a[1:5]

In [ ]:
c = @view a[1:5]

In [ ]:
no_views_sum(x) = sum(x[3:end-2])

In [ ]:
@views views_sum(x) = sum(x[3:end-2])

In [ ]:
views_sum(collect(1:5)); # compile

In [ ]:
no_views_sum(collect(1:5)); # compile

In [ ]:
a = rand(1:5, 10^6);

In [ ]:
@timev no_views_sum(a);

In [ ]:
@timev views_sum(a);

## The `LinearAlgebra` standard library

Julia also provides a very flexible set of linear algebra routines as part of its standard library. You can load these with `using LinearAlgebra` -- the stdlib comes installed with Julia itself. Under the hood, Julia is using high performance BLAS and LAPACK implementations with quite good generic fallbacks for non-BLAS types. If you have an Intel CPU, you can use Intel MKL if you wish (see the Julia documentation for how to do this). 

In [ ]:
using LinearAlgebra

In [ ]:
A = rand(ComplexF64, 4, 4)

In [ ]:
Ah = A + A' # A + adjoint(A)

In [ ]:
@which eigvals(Ah)

In [ ]:
@which eigvals(Hermitian(Ah))

Let's verify that we get the same eigenvalues no matter which path we take.

In [ ]:
eigvals(Ah)

In [ ]:
eigvals(Hermitian(Ah))

In [ ]:
A = rand(ComplexF64, 64, 64)

In [ ]:
Ah = A + A'

In [ ]:
@timev eigvals(Ah);

In [ ]:
@timev eigvals(Hermitian(Ah));